In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from cartoframes.viz import *
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
import random

/home/hj/anaconda3/envs/HJ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 초기 노드 임베딩 가져오기
def get_basic_embeddings(path, dong_name):
    emb = pd.read_csv(path)
    #emb = standardize_dataframe(emb, dong_name)   # 정규화
    features = torch.tensor(emb.iloc[:, 1:].values, dtype=torch.float32)   # 임베딩만 추출
    return emb, features

In [5]:
# 그래프 불러오기
def get_graph(path, emb, dong_name):
    with open(path, 'rb') as f:
        G = pickle.load(f)
    edges = G.edges(data=True)   # 엣지 추출
    node_to_idx = {name: idx for idx, name in enumerate(emb[dong_name])}   # 동이름 -> 인덱스 변환
    edge_index = torch.tensor([[node_to_idx[edge[0]], node_to_idx[edge[1]]] for edge in edges], dtype=torch.long).T   # edge list 재생산
    edge_weight = torch.tensor([edge[2]['weight'] for edge in edges], dtype=torch.float32)
    
    return edge_index, edge_weight

In [46]:
# humanflow 임베딩 데이터
hf_emb = pd.read_csv('../../Dataset/Raw_Embeddings/HumanFlow_Embeddings_kor.csv')
group = hf_emb.groupby(['기준일ID'])

for key, df in group:
    print(key[0])
    print(type(key[0]))
    print(df.shape)
    print(df.columns)
    emb = df
    features = torch.tensor(emb.iloc[:, 2:].values, dtype=torch.float32)
    break

201701
<class 'int'>
(424, 37)
Index(['기준일ID', '행정동코드', '총생활인구수', '남자0세부터9세생활인구수', '남자10세부터14세생활인구수',
       '남자15세부터19세생활인구수', '남자20세부터24세생활인구수', '남자25세부터29세생활인구수',
       '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '남자40세부터44세생활인구수',
       '남자45세부터49세생활인구수', '남자50세부터54세생활인구수', '남자55세부터59세생활인구수',
       '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수', '여자0세부터9세생활인구수',
       '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', '여자20세부터24세생활인구수',
       '여자25세부터29세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수',
       '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', '여자50세부터54세생활인구수',
       '여자55세부터59세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', '여자70세이상생활인구수',
       '총생활인구수_LONG_FOREIGNER', '중국인체류인구수_LONG_FOREIGNER',
       '중국외외국인체류인구수_LONG_FOREIGNER', '총생활인구수_TEMP_FOREIGNER',
       '중국인체류인구수_TEMP_FOREIGNER', '중국외외국인체류인구수_TEMP_FOREIGNER'],
      dtype='object')


In [6]:
# human flow 그래프 데이터
basic_path = '../../Dataset/Human_flow_Graph/'
start_year = 2017
start_month = 1
end_year = 2022
end_month = 7

# 경로들 확인
path_list = [
    basic_path+f"{year}{month:02d}.gpickle"
    for year in range(start_year, end_year + 1)
    for month in range(1, 13)
    if not (year == end_year and month > end_month)
]

edge_index, edge_weight = get_graph(path_list[0], emb, '행정동코드')
edge_index

NameError: name 'emb' is not defined

In [32]:
data = Data(x=features, edge_index=edge_index, edge_weight=edge_weight)

# Check for very large or very small values
print(data.x.max(), data.x.min())  # 값 범위 확인
print(data.edge_weight.max(), data.edge_weight.min())

tensor(27.3508) tensor(-1.8878)
tensor(1.) tensor(1.)


In [7]:
# GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [8]:
# GAT 모델 정의
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [9]:
# Negative Sampling 구현
def negative_sampling(edge_index, num_nodes, num_neg_samples):
    """Negative samples 생성"""
    edge_set = set(tuple(edge) for edge in edge_index.T.tolist())
    neg_samples = []
    while len(neg_samples) < num_neg_samples:
        i, j = np.random.randint(0, num_nodes, size=2)
        if i != j and (i, j) not in edge_set and (j, i) not in edge_set:
            neg_samples.append((i, j))
    return torch.tensor(neg_samples, dtype=torch.long).T

In [10]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 멀티 GPU를 사용하는 경우
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [36]:
# 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

model_gcn = GCN(data.num_node_features, 64, 35).to(device)
model_gat = GAT(data.num_node_features, 64, 35, heads=4).to(device)  # heads=4는 각 노드에서 4개의 attention heads 사용

# 최적화 방법 설정
optimizer_gcn = torch.optim.Adam(model_gcn.parameters(), lr=0.01, weight_decay=5e-4)
optimizer_gat = torch.optim.Adam(model_gat.parameters(), lr=0.01, weight_decay=5e-4)

In [11]:
def train_model(model, data, optimizer, num_epochs=200, patience=20, seed=42):
    """
    GCN 또는 GAT 모델 학습을 위한 함수 (Early Stopping 시 best_loss 기준으로 z 반환).
    
    Args:
        model (torch.nn.Module): 학습할 모델 (GCN 또는 GAT).
        data (torch_geometric.data.Data): 학습에 사용할 데이터.
        optimizer (torch.optim.Optimizer): 최적화 알고리즘.
        num_epochs (int): 학습할 에폭 수.
        patience (int): Early Stopping 기준.
        seed (int): 재현 가능한 결과를 위한 Seed 값.

    Returns:
        torch.Tensor: `best_loss` 기준으로 저장된 노드 임베딩.
    """
    # Seed 고정
    set_seed(seed)

    best_loss = float('inf')
    counter = 0
    best_z = None  # `best_loss` 시점의 임베딩 저장

    for epoch in tqdm(range(num_epochs), desc="Training Progress"):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        if isinstance(model, GAT):  # GAT 모델에는 edge_weight 사용하지 않음
            z = model(data.x, data.edge_index)
        else:  # GCN 모델에는 edge_weight 전달
            z = model(data.x, data.edge_index, data.edge_weight)

        # Positive edges
        pos_edges = data.edge_index.T
        pos_scores = (z[pos_edges[:, 0]] * z[pos_edges[:, 1]]).sum(dim=1)

        # Negative edges
        neg_edges = negative_sampling(data.edge_index, data.num_nodes, pos_edges.size(0) // 2)
        neg_scores = (z[neg_edges[0]] * z[neg_edges[1]]).sum(dim=1)

        # Loss 계산
        pos_loss = -F.logsigmoid(pos_scores).mean()
        neg_loss = -F.logsigmoid(-neg_scores).mean()
        loss = pos_loss + neg_loss

        loss.backward()
        optimizer.step()

        # Loss 출력
        tqdm.write(f"Epoch {epoch + 1}, Loss: {loss.item()}")

        # Early Stopping 로직
        if loss.item() < best_loss:
            best_loss = loss.item()  # 최적 Loss 갱신
            best_z = z.detach().cpu()  # `best_loss` 시점의 임베딩 저장
            counter = 0  # 카운터 초기화
        else:
            counter += 1

        if counter >= patience:
            tqdm.write(f"Early stopping at epoch {epoch + 1}")
            break

    return best_z  # 최적 Loss 기준의 임베딩 반환

In [41]:
z_gcn = train_model(model_gcn, data, optimizer_gcn, num_epochs=200, patience=20)
z_gat = train_model(model_gat, data, optimizer_gat, num_epochs=200, patience=20)

Training Progress:   1%|          | 2/200 [00:00<01:23,  2.36it/s]

Epoch 1, Loss: 2.1354637145996094
Epoch 2, Loss: 1.115706443786621


Training Progress:   2%|▏         | 3/200 [00:01<00:56,  3.51it/s]

Epoch 3, Loss: 1.0097235441207886


Training Progress:   2%|▎         | 5/200 [00:01<00:39,  4.91it/s]

...

Epoch 186, Loss: 0.28905385732650757


Training Progress:  94%|█████████▍| 188/200 [00:24<00:01,  7.29it/s]

Epoch 187, Loss: 0.21819478273391724
Epoch 188, Loss: 0.21753206849098206


Training Progress:  94%|█████████▍| 188/200 [00:25<00:01,  7.51it/s]

Epoch 189, Loss: 0.20613722503185272
Early stopping at epoch 189


In [42]:
def save_embeddings(z, emb, output_path, model_name="model"):
    """
    학습된 임베딩을 CSV 파일로 저장하는 함수.

    Args:
        z (torch.Tensor): 학습된 임베딩 (노드 임베딩).
        emb (pd.DataFrame): 원본 데이터프레임 (ADM_NM 열 포함).
        output_path (str): 저장할 CSV 파일 경로.
        model_name (str): 임베딩 컬럼에 추가할 모델 이름 접두사 (예: "GCN", "GAT").

    Returns:
        None
    """
    # GPU 텐서를 NumPy 배열로 변환
    z_numpy = z.detach().cpu().numpy()

    # 새로운 데이터프레임 생성: ADM_NM + 학습된 임베딩
    embedding_columns = [f"{model_name}_embed{i}" for i in range(z_numpy.shape[1])]  # 임베딩 컬럼 이름 생성
    emb_updated = pd.DataFrame(emb[['기준일ID', '행정동코드']], columns=['기준일ID', '행정동코드'])  # ADM_NM 열만 복사
    emb_updated[embedding_columns] = z_numpy  # 임베딩 추가

    # CSV로 저장
    # emb_updated.to_csv(output_path, index=False)
    # print(f"Embeddings saved to: {output_path}")
    
    return emb_updated

In [43]:
gcn_df = save_embeddings(z_gcn, emb, 'a', 'GCN')
gat_df = save_embeddings(z_gat, emb, 'b', 'GAT')

In [44]:
gcn_df

,기준일ID,행정동코드,GCN_embed0,GCN_embed1,GCN_embed2,GCN_embed3,GCN_embed4,GCN_embed5,GCN_embed6,GCN_embed7,...,GCN_embed25,GCN_embed26,GCN_embed27,GCN_embed28,GCN_embed29,GCN_embed30,GCN_embed31,GCN_embed32,GCN_embed33,GCN_embed34
0,201701,청운효자동,-1.074775,1.010859,1.665435,-2.015037,-1.024539,0.843725,-1.007559,-1.663782,...,1.743431,1.089767,2.082939,-0.009981,0.455670,0.220273,-2.229680,-0.264066,0.534541,2.462892
1,201701,사직동,-0.956690,0.717311,1.137805,-1.975991,-0.219769,0.996532,-1.062324,-1.641588,...,1.297022,0.875547,1.785869,-0.013894,0.423590,0.208445,-2.219543,-0.272838,0.242694,2.039333
2,201701,삼청동,-1.016063,0.709976,1.073384,-2.374769,-0.114190,1.251863,-1.170677,-1.878487,...,1.424986,0.841455,1.936456,-0.118748,0.536596,0.226169,-2.537327,-0.285981,0.184181,2.246561
3,201701,부암동,-1.132262,0.798428,1.176160,-2.667495,-0.206448,1.414158,-1.250911,-2.084862,...,1.627175,0.877967,2.139970,-0.214334,0.592086,0.264167,-2.791227,-0.348340,0.243941,2.499584
4,201701,평창동,-1.210514,0.861963,1.259817,-2.804583,-0.314177,1.484705,-1.290055,-2.185333,...,1.761885,0.901824,2.261122,-0.268917,0.613923,0.283362,-2.898780,-0.383092,0.298177,2.636966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,201701,성내2동,0.619395,0.045704,1.792655,0.608224,5.028989,0.806009,-1.161979,-1.452925,...,-1.510692,1.862023,-0.171477,-1.740461,-3.546756,0.678891,-1.370561,-1.721826,1.257090,1.731453
420,201701,성내3동,0.620451,0.043403,1.797358,0.623784,5.049545,0.804108,-1.163838,-1.451354,...,-1.513293,1.869788,-0.174239,-1.755055,-3.558245,0.676727,-1.386368,-1.731173,1.251065,1.733127
421,201701,길동,0.621935,0.040838,1.802128,0.638617,5.072171,0.802370,-1.165782,-1.449877,...,-1.516407,1.877763,-0.177339,-1.770049,-3.570510,0.674650,-1.403128,-1.740887,1.244996,1.735240
422,201701,둔촌1동,0.623400,0.038370,1.806558,0.652348,5.093230,0.800713,-1.167281,-1.448273,...,-1.519340,1.885019,-0.180316,-1.784241,-3.581704,0.672697,-1.418999,-1.750064,1.239304,1.737058


# AirBnB 데이터 다시 저장

In [55]:
import ast  # 문자열 리스트를 실제 리스트로 변환

def expand_llm_embeddings(df, embeddings_col='LLM Embeddings'):
    """
    주어진 데이터프레임의 LLM Embeddings 열을 확장하여 각 값을 개별 열로 변환.
    
    Parameters:
        df (pd.DataFrame): 입력 데이터프레임.
        embeddings_col (str): 확장할 열의 이름 (기본값: 'LLM Embeddings').

    Returns:
        pd.DataFrame: LLM Embeddings가 확장된 데이터프레임.
    """
    # Step 1: LLM Embeddings를 리스트로 변환
    df[embeddings_col] = df[embeddings_col].apply(ast.literal_eval)

    # Step 2: 리스트 길이 확인
    max_len = max(len(x) for x in df[embeddings_col])
    print(f"Max length of {embeddings_col}: {max_len}")

    # Step 3: LLM Embeddings를 확장하여 새 열 생성
    embedding_columns = [f"{embeddings_col}_{i}" for i in range(max_len)]
    embedding_df = pd.DataFrame(df[embeddings_col].tolist(), columns=embedding_columns, index=df.index)

    # Step 4: 기존 데이터프레임과 병합
    df = pd.concat([df, embedding_df], axis=1)

    # Step 5: 기존 LLM Embeddings 열 제거
    df = df.drop(columns=[embeddings_col])

    return df

In [59]:
# 데이터 로드
emb = pd.read_csv('../../Dataset/AirBnB_LLM/gemma2_no_listing_raw_fill_na_ver.csv')
#emb = emb[['ADM_NM', 'Reporting Month', 'LLM Embeddings']]
#emb = expand_llm_embeddings(emb, embeddings_col='LLM Embeddings')

emb

,ADM_NM,Reporting Month,Property ID,Occupancy Rate,Revenue (USD),Number of Reservations,LLM Embeddings
0,가락1동,2017-01-01,"['ab-8296118', 'ab-13633415', 'ab-14432019', '...",0.000000,0.00,0.0,"['0.15136719', '-0.13867188', '-1.2421875', '-..."
1,가락2동,2017-01-01,"['ab-6240301', 'ab-14455385', 'ab-2039222']",0.000000,0.00,0.0,"['0.20410156', '-0.1953125', '-1.234375', '-0...."
2,가락본동,2017-01-01,"['ab-7607282', 'ab-7792221', 'ab-7792395', 'ab...",0.249571,4319.67,30.0,"['0.24902344', '-0.17675781', '-1.2421875', '-..."
3,가리봉동,2017-01-01,"['ab-7538039', 'ab-11981624', 'ab-11987983', '...",0.200000,515.00,5.0,"['0.36914062', '-0.20996094', '-1.203125', '-0..."
4,가산동,2017-01-01,"['ab-9098630', 'ab-10234965', 'ab-11777964', '...",0.000000,0.00,0.0,"['0.31445312', '-0.24121094', '-1.0546875', '-..."
...,...,...,...,...,...,...,...
28403,효창동,2022-07-01,"['ab-53466085', 'ab-54039869', 'ab-59300548353...",0.209875,8802.74,44.0,"['0.15722656', '-0.29101562', '-1.4140625', '-..."
28404,후암동,2022-07-01,"['ab-660505188610948586', 'ab-6620696254432664...",0.292348,113240.44,387.0,"['0.39453125', '-0.375', '-1.234375', '-0.2773..."
28405,휘경1동,2022-07-01,"['ab-661330943863214668', 'ab-52644575', 'ab-5...",0.348942,18969.15,154.0,"['0.18457031', '-0.38085938', '-1.4453125', '-..."
28406,휘경2동,2022-07-01,"['ab-53056242', 'ab-593955664612254971', 'ab-4...",0.268609,4532.20,41.0,"['0.18652344', '-0.31054688', '-1.3515625', '-..."


In [12]:
import os

basic_path = '../../Dataset/AirBnB_LLM'
output_path = '../../Dataset/AirBnB_LLM_New'

# 파일명들 불러오기
file_names = os.listdir(basic_path)
llm_list = [s for s in file_names  if '_fill_na_ver' in s]
llm_list

['llama3_no_listing_refined_fill_na_ver.csv',
 'llama3_mixed_refined_fill_na_ver.csv',
 'llama3_mixed_raw_fill_na_ver.csv',
 'llama3_only_listings_fill_na_ver.csv',
 'gemma2_no_listing_raw_fill_na_ver.csv',
 'gemma2_no_listing_refined_fill_na_ver.csv',
 'llama3_no_listing_raw_fill_na_ver.csv']

In [64]:
# llm 데이터 다시 저장
for llm in llm_list:
    emb = pd.read_csv(f'{basic_path}/{llm}')
    labels = emb[['ADM_NM', 'Reporting Month', 'Occupancy Rate', 'Revenue (USD)', 'Number of Reservations']]  # 라벨변수
    emb = emb[['ADM_NM', 'Reporting Month', 'LLM Embeddings']]
    emb = expand_llm_embeddings(emb, embeddings_col='LLM Embeddings')
    emb.to_csv(f'{output_path}/{llm}', index=False)   # 새롭게 저장
    labels.to_csv(f'{output_path}/{llm[:-4]}_labels.csv', index=False)
    print(f'{llm} finish!')
    print(emb.shape)

Max length of LLM Embeddings: 3072
llama3_no_listing_refined_fill_na_ver.csv finish!
(28408, 3074)
Max length of LLM Embeddings: 3072
llama3_mixed_refined_fill_na_ver.csv finish!
(28408, 3074)
Max length of LLM Embeddings: 3072
llama3_mixed_raw_fill_na_ver.csv finish!
(28408, 3074)
Max length of LLM Embeddings: 3072
llama3_only_listings_fill_na_ver.csv finish!
(28408, 3074)
Max length of LLM Embeddings: 2304
gemma2_no_listing_raw_fill_na_ver.csv finish!
(28408, 2306)
Max length of LLM Embeddings: 2304
gemma2_no_listing_refined_fill_na_ver.csv finish!
(28408, 2306)
Max length of LLM Embeddings: 3072
llama3_no_listing_raw_fill_na_ver.csv finish!
(28408, 3074)


In [16]:
# 완성 버전으로 불러오기

basic_path = '../../Dataset/AirBnB_LLM_New/'

# 파일명들 불러오기
file_names = os.listdir(basic_path)
llm_list = [s for s in file_names  if '_fill_na_ver.csv' in s]
llm_list

['llama3_no_listing_refined_fill_na_ver.csv',
 'llama3_mixed_refined_fill_na_ver.csv',
 'llama3_mixed_raw_fill_na_ver.csv',
 'llama3_only_listings_fill_na_ver.csv',
 'gemma2_no_listing_raw_fill_na_ver.csv',
 'gemma2_no_listing_refined_fill_na_ver.csv',
 'llama3_no_listing_raw_fill_na_ver.csv']

In [ ]:
# 새롭게 저장된 버전 불러오기
llm_df = pd.read_csv(basic_path+llm_list[0])
llm_df

,ADM_NM,Reporting Month,LLM Embeddings_0,LLM Embeddings_1,LLM Embeddings_2,LLM Embeddings_3,LLM Embeddings_4,LLM Embeddings_5,LLM Embeddings_6,LLM Embeddings_7,...,LLM Embeddings_3062,LLM Embeddings_3063,LLM Embeddings_3064,LLM Embeddings_3065,LLM Embeddings_3066,LLM Embeddings_3067,LLM Embeddings_3068,LLM Embeddings_3069,LLM Embeddings_3070,LLM Embeddings_3071
0,가락1동,2017-01-01,0.182189,0.797635,2.783483,-0.701678,0.322488,-0.958251,-0.049312,0.477725,...,-0.097656,0.215152,0.293026,-0.321801,0.168507,0.383450,1.076234,-0.122758,-0.496166,0.353151
1,가락2동,2017-01-01,0.012297,0.788067,2.737332,-0.818893,0.190079,-1.046540,-0.118247,0.573178,...,-0.084388,0.256985,0.328190,-0.304241,0.251695,0.308751,0.974102,-0.033396,-0.474156,0.405358
2,가락본동,2017-01-01,-0.188756,0.613868,2.676397,-0.363336,0.272512,-1.133036,0.080491,0.709784,...,-0.180724,0.597994,0.214895,-0.032563,0.046565,0.770207,0.793798,0.051106,-0.697922,0.204088
3,가리봉동,2017-01-01,-0.019578,0.852448,2.604572,-0.697828,0.285646,-0.979806,0.017082,0.709763,...,-0.226313,0.189473,0.370528,-0.330595,0.197076,0.346764,0.979650,0.030385,-0.429923,0.396902
4,가산동,2017-01-01,-0.112694,0.606479,2.593920,-0.665623,0.441548,-1.082860,-0.001558,0.557418,...,-0.176513,0.336681,0.450447,-0.124183,0.145434,0.386809,1.054443,0.105295,-0.655455,0.193765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28403,효창동,2022-07-01,-0.057492,0.588885,2.498071,-0.576995,0.077653,-1.097834,0.177970,0.717874,...,-0.014342,0.495624,0.320659,0.046189,-0.013092,0.547956,0.944268,0.102939,-0.605520,0.238395
28404,후암동,2022-07-01,-0.113301,0.605343,2.413508,-0.357536,0.357129,-0.846870,-0.298720,0.749327,...,-0.429303,0.589567,0.152206,0.046052,-0.068495,0.537594,0.725969,0.042671,-0.721411,0.627279
28405,휘경1동,2022-07-01,0.002187,0.594431,2.574777,-0.520903,0.230063,-0.997220,0.003344,0.779653,...,-0.275357,0.383240,0.274320,0.014191,0.064286,0.339190,1.015625,0.167795,-0.624249,0.414327
28406,휘경2동,2022-07-01,0.048968,0.679767,2.465750,-0.614556,0.219916,-1.036563,0.116839,0.775792,...,-0.144036,0.361136,0.269136,0.003099,0.118099,0.432616,0.995827,0.153141,-0.507071,0.349329


In [24]:
group = llm_df.groupby(['Reporting Month'])
for key, df in group:
    print(str(key[0]).replace('-',''))
    display(df)
    break

20170101


,ADM_NM,Reporting Month,LLM Embeddings_0,LLM Embeddings_1,LLM Embeddings_2,LLM Embeddings_3,LLM Embeddings_4,LLM Embeddings_5,LLM Embeddings_6,LLM Embeddings_7,...,LLM Embeddings_3062,LLM Embeddings_3063,LLM Embeddings_3064,LLM Embeddings_3065,LLM Embeddings_3066,LLM Embeddings_3067,LLM Embeddings_3068,LLM Embeddings_3069,LLM Embeddings_3070,LLM Embeddings_3071
0,가락1동,2017-01-01,0.182189,0.797635,2.783483,-0.701678,0.322488,-0.958251,-0.049312,0.477725,...,-0.097656,0.215152,0.293026,-0.321801,0.168507,0.383450,1.076234,-0.122758,-0.496166,0.353151
1,가락2동,2017-01-01,0.012297,0.788067,2.737332,-0.818893,0.190079,-1.046540,-0.118247,0.573178,...,-0.084388,0.256985,0.328190,-0.304241,0.251695,0.308751,0.974102,-0.033396,-0.474156,0.405358
2,가락본동,2017-01-01,-0.188756,0.613868,2.676397,-0.363336,0.272512,-1.133036,0.080491,0.709784,...,-0.180724,0.597994,0.214895,-0.032563,0.046565,0.770207,0.793798,0.051106,-0.697922,0.204088
3,가리봉동,2017-01-01,-0.019578,0.852448,2.604572,-0.697828,0.285646,-0.979806,0.017082,0.709763,...,-0.226313,0.189473,0.370528,-0.330595,0.197076,0.346764,0.979650,0.030385,-0.429923,0.396902
4,가산동,2017-01-01,-0.112694,0.606479,2.593920,-0.665623,0.441548,-1.082860,-0.001558,0.557418,...,-0.176513,0.336681,0.450447,-0.124183,0.145434,0.386809,1.054443,0.105295,-0.655455,0.193765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,효창동,2017-01-01,-0.209827,0.555203,2.690814,-0.383771,0.146396,-1.003996,-0.103601,0.638757,...,-0.072862,0.744518,0.174405,-0.045266,0.099417,0.663327,0.911730,-0.033662,-0.775468,0.109442
420,후암동,2017-01-01,-0.162344,0.632814,2.539969,-0.355085,0.503987,-0.800747,-0.469939,0.773364,...,-0.388491,0.642842,0.226201,-0.048449,0.059380,0.547762,0.668395,0.119956,-0.755786,0.512600
421,휘경1동,2017-01-01,-0.347272,0.624385,2.570642,-0.407633,0.262688,-1.072827,0.171972,0.756842,...,-0.226987,0.545958,0.381583,-0.023453,0.094286,0.634946,0.807298,-0.003554,-0.688187,0.353662
422,휘경2동,2017-01-01,-0.198812,0.617473,2.636014,-0.380618,0.269563,-0.991828,0.192307,0.750091,...,-0.194521,0.533942,0.319364,0.139239,0.148130,0.393189,1.117845,0.114787,-0.543678,0.152803


In [20]:
# airbnb 그래프 구조 불러오기
basic_path = '../../Dataset/AirBnB_Graph/llama3_no_listing_raw/'
start_year = 2017
start_month = 1
end_year = 2022
end_month = 7

# 경로들 확인
path_list = [
    basic_path+f"{year}{month:02d}01.gpickle"
    for year in range(start_year, end_year + 1)
    for month in range(1, 13)
    if not (year == end_year and month > end_month)
]

edge_index, edge_weight = get_graph(path_list[0], llm_df, 'ADM_NM')
edge_index

tensor([[27984, 27984, 27984,  ..., 28405, 28405, 28406],
        [27988, 27990, 27997,  ..., 28406, 28407, 28407]])